In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores.chroma import Chroma
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
import os
import glob


import os
import os
# Set the OPENAI_API_KEY environment variable
os.environ["OPENAI_API_KEY"] = (
    "sk-proj-xxxxxxxxxxxxxxxxxxxxxxxxxxxx"
)

# Initialize embeddings
embeddings = OpenAIEmbeddings()

# Define text splitter for chunking
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=1000,
    chunk_overlap=100
)

In [2]:

# Directory containing multiple PDF files
pdf_directory = "./data/"
# Load all PDF documents from directory
documents = []
for pdf_path in glob.glob(os.path.join(pdf_directory, "*.pdf")):
    loader = PyPDFLoader(pdf_path)
    pdf_documents = loader.load_and_split(text_splitter=text_splitter)
    documents.extend(pdf_documents)



In [3]:
len(documents)

75

In [ ]:
for d in documents:
    print(d.metadata)
    print(d.page_content)
    print('###########################')

{'source': '/content/sample_data/pdf/BATTERY AND COMPONENT.pdf', 'page': 0}
HV BATTERY SYSTEM 
Temperature Limits 
Battery pack and vehicle can operate  
safely in limits from 0°C to 50°C. 
Hv Battery Life & Maintenance 
This Vehicle comes with a battery life of  
eight years or 160,000km warranty for the 
battery, whichever is earlier. Regular serv-
ice of the vehicle and charging protocol to 
be followed to maximize the battery life. 
Energy Information 
The vehicle battery pack has a maximum  
energy of 40.5kWh. Energy retention ca -
pacity deteriorates over several cycles of  
usage and hence range deterioration hap-
pens overtime.  
This decrease in range during the end of  
life of battery is expected and is not con-
sidered as a malfunction of the battery  
pack. During these conditions, it is recom-
mended to contact an authorized service  
center for inspection. 
Brake Energy Recovery System 
The vehicle features energy regeneration 
system, which regenerates expended en-
ergy

In [4]:
# Initialize and store embeddings in ChromaDB
db = Chroma.from_documents(
    documents,
    embedding=embeddings,
    persist_directory="emb5"
)

In [ ]:
# Perform similarity search
results = db.similarity_search(
    "What are the main differences between electric and conventional vehicles?",
    k=1
)

In [ ]:
results

[Document(metadata={'page': 0, 'source': '/content/sample_data/pdf/INTRODUCTION TO EV.pdf'}, page_content='EV OVERVIEW \nAn electric vehicle is powered by a battery \n- Edrive and it does not need any type of\nfossil fuel. \nWhile conventional vehicles use\nan internal \ncombustion engine and gaso -\nline or diesel as fuel, electric vehicles use\nelectrical energy \nthat is stored inside the\nhigh voltage \nbattery. As a result, electric\nvehicles run \non electricity, they are eco-\nfriendly - they do not require fuel and are\nzero emission vehicles.\nReview and Characteristics \nThis is an electric vehicle. Some of the ve-\nhicle’s systems operate differently and  \nhave different operating characteristics  \nthan vehicles equipped with an internal  \ncombustion engine. It is important to care-\nfully read the entire Owner’s Manual for  \nthis reason. \nThis Vehicle uses two types of battery sys-\ntems- a high voltage system in which a  \nhigh voltage battery powers the inverter  \na

In [ ]:
for r in results:
    print('Page No :',r.metadata['page'])
    print('Source :',r.metadata['source'])
    print('page_content :',r.page_content)

Page No : 0
Source : /content/sample_data/pdf/INTRODUCTION TO EV.pdf
page_content : EV OVERVIEW 
An electric vehicle is powered by a battery 
- Edrive and it does not need any type of
fossil fuel. 
While conventional vehicles use
an internal 
combustion engine and gaso -
line or diesel as fuel, electric vehicles use
electrical energy 
that is stored inside the
high voltage 
battery. As a result, electric
vehicles run 
on electricity, they are eco-
friendly - they do not require fuel and are
zero emission vehicles.
Review and Characteristics 
This is an electric vehicle. Some of the ve-
hicle’s systems operate differently and  
have different operating characteristics  
than vehicles equipped with an internal  
combustion engine. It is important to care-
fully read the entire Owner’s Manual for  
this reason. 
This Vehicle uses two types of battery sys-
tems- a high voltage system in which a  
high voltage battery powers the inverter  
and electric motor which in turn propel the 
vehicl

In [ ]:
# Similarity search with score
results_with_scores = db.similarity_search_with_score(
    "What are the main differences between electric and conventional vehicles?"
)

In [ ]:
results_with_scores

[(Document(metadata={'page': 0, 'source': '/content/sample_data/pdf/INTRODUCTION TO EV.pdf'}, page_content='EV OVERVIEW \nAn electric vehicle is powered by a battery \n- Edrive and it does not need any type of\nfossil fuel. \nWhile conventional vehicles use\nan internal \ncombustion engine and gaso -\nline or diesel as fuel, electric vehicles use\nelectrical energy \nthat is stored inside the\nhigh voltage \nbattery. As a result, electric\nvehicles run \non electricity, they are eco-\nfriendly - they do not require fuel and are\nzero emission vehicles.\nReview and Characteristics \nThis is an electric vehicle. Some of the ve-\nhicle’s systems operate differently and  \nhave different operating characteristics  \nthan vehicles equipped with an internal  \ncombustion engine. It is important to care-\nfully read the entire Owner’s Manual for  \nthis reason. \nThis Vehicle uses two types of battery sys-\ntems- a high voltage system in which a  \nhigh voltage battery powers the inverter  \n

In [ ]:
for document, score in results_with_scores:
    page = document.metadata.get("page", "N/A")
    source = document.metadata.get("source", "N/A")
    page_content = document.page_content

    print(f"Page: {page}")
    print(f"Source: {source}")
    print(f"Content: {page_content}")
    print(f"Similarity Score: {score}")
    print("\n" + "-" * 40 + "\n")

Page: 0
Source: /content/sample_data/pdf/INTRODUCTION TO EV.pdf
Content: EV OVERVIEW 
An electric vehicle is powered by a battery 
- Edrive and it does not need any type of
fossil fuel. 
While conventional vehicles use
an internal 
combustion engine and gaso -
line or diesel as fuel, electric vehicles use
electrical energy 
that is stored inside the
high voltage 
battery. As a result, electric
vehicles run 
on electricity, they are eco-
friendly - they do not require fuel and are
zero emission vehicles.
Review and Characteristics 
This is an electric vehicle. Some of the ve-
hicle’s systems operate differently and  
have different operating characteristics  
than vehicles equipped with an internal  
combustion engine. It is important to care-
fully read the entire Owner’s Manual for  
this reason. 
This Vehicle uses two types of battery sys-
tems- a high voltage system in which a  
high voltage battery powers the inverter  
and electric motor which in turn propel the 
vehicle and a low

In [5]:
# Build retrieval chain

# step 1 :This line initializes a ChatOpenAI object
chat = ChatOpenAI()


#step 2 : This line assumes you have a database (db) that has already been set up
# The .as_retriever() method turns your database into a retriever object.
# A retriever's job is to find relevant information within the database based on a given query.

retriever = db.as_retriever( )

# Step 3 : It creates a RetrievalQA chain using the from_chain_type method.
# This chain combines the LLM (chat) with the retriever (retriever) to answer questions based on the
# information in your database.
chain = RetrievalQA.from_chain_type(
    llm=chat,
    retriever=retriever,
    chain_type="stuff"
)

# Note : chain_type="stuff": This defines the type of chain to use.
# The "stuff" chain type means that the retriever will fetch relevant documents
# and the LLM will receive all the retrieved documents as context to answer the question.




In [6]:
# Execute the retrieval chain
result = chain.invoke("What are the main differences between electric and conventional vehicles?")
print(result['result'])

The main differences between electric vehicles and conventional vehicles are:

1. Power Source: Electric vehicles are powered by batteries (Edrive) that store electrical energy, while conventional vehicles run on gasoline or diesel with an internal combustion engine.

2. Emissions: Electric vehicles are eco-friendly and zero-emission vehicles, as they do not produce exhaust gases like conventional vehicles, contributing to reduced air pollution.

3. Components: Electric vehicles have different components like an electric motor and high voltage battery, whereas conventional vehicles have an internal combustion engine.

4. Charging: Electric vehicles need to be charged with electricity before driving, and they can be recharged by plugging in. Conventional vehicles require filling up with gasoline or diesel at a gas station.

5. Driving Range: Electric vehicles may have different driving ranges compared to conventional vehicles due to battery technology and charging infrastructure differe

In [ ]:
### Advanced Approach

User Interface For RAG

In [9]:
import gradio as gr

In [10]:
def answer_question(question,history):
    result = chain.invoke(question)
    return result['result']

In [14]:
# ChatInterface does not need inputs/outputs
iface = gr.ChatInterface(
    fn=answer_question,
    title="Smart Electric Vehicle Assistant",
    description="Ask your query",
)

iface.launch()

c:\Users\AbhishekShrivastav\Desktop\Allinz-1\labenv1\lib\site-packages\gradio\chat_interface.py:345: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.
